# Wissensaggregator Mittelalter und frühe Neuzeit
### Daten für Klöster und Domstifte einlesen

Quelle: Datenbank `gs_klosterdatenbank`, siehe Freigabe https://owncloud.gwdg.de/index.php/s/s4x0TotkZq4Nm2B

Die Einträge in der Tabelle `person_role` beziehen sich auf diese Daten.

- [Einträge in `item`](#Einträge-in-item)
- [Einträge in `institution`](#Einträge-in-institution)
- [Einträge in `institution_place`](#Einträge-in-institution_place)

In [1]:
# wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"
wds_path="..\\.."

"..\\.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using DataFrames

In [8]:
Wds = WiagDataSetup

WiagDataSetup

*2022-11-04* Fix für Domstift Verden mit Datenbank `wiag_in`

In [9]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

In [10]:
item_type_id = 2 # Kloster

2

### Einträge in `item`

entsprechende Einträge in Tabelle `item` einfügen

`id` für 'Kloster' in `item_type`: 2

Einträge löschen (Korrektur/Datenupdate)



In [10]:
item_type_mon = 2; # Kloster
item_type_dft = 3; # Domstift

In [117]:
table_name = "institution_place"
sql = "DELETE from $(table_name) WHERE institution_id in 
(SELECT id FROM item WHERE item_type_id in ($(item_type_mon), $item_type_dft))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE from institution_place WHERE institution_id in \n(SELECT id FROM item WHERE item_type_id in (2, 3))", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [62]:
table_name = "institution"
sql = "DELETE from $(table_name) WHERE id in 
(SELECT id FROM item WHERE item_type_id in ($(item_type_mon), $item_type_dft))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE from institution WHERE id in \n(SELECT id FROM item WHERE item_type_id in (2, 3))", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [14]:
table_name = "item"
sql = "DELETE FROM $(table_name) WHERE item_type_id in ($(item_type_mon), $item_type_dft)"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM item WHERE item_type_id in (2, 3)", 0, -1, 5286, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Beschränkung auf `status = Online`, da die Redaktion außerhalb von WIAG liegt

In [14]:
sql = "SELECT * FROM gs_klosterdatenbank.gs_monastery WHERE status = 'Online'"
df_mon = Wds.sql_df(sql);

In [15]:
Wds.clean_up!(df_mon)

In [16]:
size(df_mon)

(5286, 15)

In [99]:
names(df_mon)

15-element Vector{String}:
 "id_monastery"
 "id_gsn"
 "date_created"
 "created_by_user"
 "status"
 "note"
 "monastery_name"
 "patrocinium"
 "selection"
 "processing_status"
 "gs_persons"
 "selection_criteria"
 "last_change"
 "changed_by_user"
 "founder"

In [28]:
columns = [
    :id_monastery => :id_in_source,
    :id_gsn => :id_gsn,
]

2-element Vector{Pair{Symbol, Symbol}}:
 :id_monastery => :id_in_source
       :id_gsn => :id_gsn

In [31]:
df_item = select(df_mon, columns);

In [19]:
using Dates

In [20]:
user_wiag_id = 1;
time_stamp = Dates.format(now(), dateformat"yyyy-mm-dd HH:MM");

In [32]:
insertcols!(
    df_item, 
    3,
    :item_type_id => item_type_id,
    :edit_status => "importiert",
    :created_by => user_wiag_id,
    :date_created => time_stamp,
    :changed_by => user_wiag_id,
    :date_changed => time_stamp,
    :is_online => 1
);

In [33]:
df_item[20:27, [:id_in_source, :id_gsn, :date_changed, :item_type_id]]

,id_in_source,id_gsn,date_changed,item_type_id
,Int32,Int32,String,Int64
1,21,75,2022-01-18 07:57,2
2,22,78,2022-01-18 07:57,2
3,23,81,2022-01-18 07:57,2
4,24,82,2022-01-18 07:57,2
5,25,84,2022-01-18 07:57,2
6,26,85,2022-01-18 07:57,2
7,27,86,2022-01-18 07:57,2
8,28,87,2022-01-18 07:57,2


In [26]:
size(df_item)

(5286, 8)

`item_type.id = 3` für Domstifte

In [24]:
gs_ids_dft = (
    226, 343, 616, 628, 675, 676, 679, 701, 736, 739, 783, 792,
    794, 803, 832, 953, 2066, 3487, 3488, 3489, 3490, 3491, 3492, 3493,
    3494, 3495, 3496, 3498, 3499, 3500, 3501, 3502, 3503, 60130
)

(226, 343, 616, 628, 675, 676, 679, 701, 736, 739, 783, 792, 794, 803, 832, 953, 2066, 3487, 3488, 3489, 3490, 3491, 3492, 3493, 3494, 3495, 3496, 3498, 3499, 3500, 3501, 3502, 3503, 60130)

In [26]:
is_id_dft(id) = id in gs_ids_dft

is_id_dft (generic function with 1 method)

In [35]:
df_item_dft = filter(:id_gsn => is_id_dft, df_item, view = true);

In [37]:
df_item_dft[:, :item_type_id] .= item_type_dft;

In [39]:
table_name = "item";
Wds.filltable!(table_name, select(df_item, Not(:id_gsn)))

┌ Info: Rows inserted: 5286
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


5286

Eintrag von `domstift_short` in `item_property`  
*2022-03-14* TODO Die Kurzfassung des Namens ist essentiell für Facetten und die Abfrage der Domherren, daher übernehmen wir ihn direkt in die Tabelle `institution`

SQL
```sql
SELECT i.id, id_gsn, name
FROM institution as inst
JOIN item as i on inst.id = i.id
WHERE i.item_type_id = 3;
```

In [11]:
df_iprop = DataFrame(
    item_id = [
        79558, 79663, 79825, 79835, 79892, 79908, 79909, 79910, 79921, 79924,
        79983, 79989, 79991, 79998, 80025, 80106, 80300, 80633, 80634, 80635,
        80636, 80637, 80638, 80639, 80640, 80641, 80642, 80643, 80644, 80645,
        80646, 80647, 80648, 82344
    ],
    name = fill("domstift_short", 34),
    value = [
        "Ratzeburg", "Bremen", "Münster", "Minden", "Brandenburg", "Schleswig", "Lübeck", "Hamburg", "Havelberg", "Lebus",
        "Osnabrück", "Merseburg", "Meißen", "Halberstadt", "Hildesheim", "Paderborn", "Schwerin", "Köln", "Würzburg", "Mainz",
        "Trier", "Konstanz", "Augsburg", "Freising", "Passau", "Regensburg", "Eichstätt", "Bamberg", "Naumburg", "Worms",
        "Speyer", "Magdeburg", "Verden", "Chiemsee"
    ]
);

In [13]:
table_name = "item_property"
Wds.filltable!(table_name, df_iprop, clear_table = false);

┌ Info: Rows inserted: 34
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


### Einträge in `institution`

Indices auslesen

In [10]:
item_type_mon = 2;
item_type_dft = 3;

In [11]:
table_name = "item";
sql = "SELECT id, id_in_source, item_type_id FROM $(table_name)
WHERE item_type_id in ($(item_type_mon), $(item_type_dft))";
df_idx = Wds.sql_df(sql);

In [12]:
columns = [
    :id_monastery => :id_in_source,
    :id_gsn => :id_gsn,
    :monastery_name => :name,
    :note => :note
]

4-element Vector{Pair{Symbol, Symbol}}:
   :id_monastery => :id_in_source
         :id_gsn => :id_gsn
 :monastery_name => :name
           :note => :note

In [17]:
df_mon_db = select(df_mon, columns);

In [18]:
transform!(df_mon_db, :id_in_source => ByRow(string) => :id_in_source);

In [19]:
df_mon_db = leftjoin(df_mon_db, df_idx, on = :id_in_source);

In [31]:
size(df_mon_db)

(5286, 6)

*2022-03-14* TODO Kurzform des Namens zuordnen (siehe oben) und direkt in Tabelle `institution` eintragen.

In [63]:
table_name = "institution";
Wds.filltable!(table_name, select(df_mon_db, Not(:id_in_source)), clear_table = false)

┌ Info: Rows inserted: 5286
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


5286

### Einträge in `institution_place`

In [21]:
table_name = "gs_klosterdatenbank.gs_monastery_location";
sql = "SELECT * FROM $(table_name) WHERE place_id IS NOT NULL";
df_inst_place = Wds.sql_df(sql);

In [22]:
Wds.clean_up!(df_inst_place)

`place.id` zuordnen

In [23]:
table_name = "gs_klosterdatenbank.gs_places";
sql = "select id_places, geonames_id FROM $(table_name) 
WHERE geonames_id IS NOT NULL AND id_places IS NOT NULL";
df_geonames = Wds.sql_df(sql);

In [24]:
df_inst_place = innerjoin(df_inst_place, df_geonames, on = :place_id => :id_places);

In [32]:
table_name = "place"
sql = "SELECT id, geonames_id, name as place_name FROM $(table_name)"
df_place = Wds.sql_df(sql);

In [34]:
df_inst_place = leftjoin(df_inst_place, df_place, on = :geonames_id, matchmissing = :notequal);

In [35]:
size(df_inst_place)

(6343, 19)

*2022-11-04* debug begin Domstift Verden

In [42]:
df_place_verden = subset(df_place, :geonames_id => ByRow(isequal(6557457)))

0×3 DataFrame

In [37]:
df_verden = subset(df_inst_place, :gsn_id => ByRow(isequal(3487)));

In [40]:
info_columns = [
    :id_monastery_location, 
    :place_id, 
    :gsn_id, 
    :location_begin_tpq, 
    :location_end_tpq, 
    :location_name,
    :geonames_id,
    :place_name
];
df_verden[:, info_columns]

Row,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_end_tpq,location_name,geonames_id,place_name
,Int32,Int32,Int32,Int32?,Int32?,String?,Int32?,String?
1,1892,12105,3487,840,1648,Verden,6557457,missing


Unter dieser geonames_id haben wir keinen Eintrag in `place`, weil diese geoname_id sich auf den Eintrag mit Geonames Feature Code A, ADM4 bezieht, während wir nur Feature Code P ausgelesen haben.

debug end

In [36]:
size(df_mon_db)

(5286, 6)

In [40]:
names(df_mon_db)

6-element Vector{String}:
 "id_in_source"
 "id_gsn"
 "name"
 "note"
 "id"
 "item_type_id"

Den WIAG-Institutionen zuordnen

In [41]:
is_id_mon(id) = id in df_mon_db.id_gsn

is_id_mon (generic function with 1 method)

In [42]:
columns = [
    :id_gsn => :id_gsn,
    :id => :institution_id,
]

2-element Vector{Pair{Symbol, Symbol}}:
 :id_gsn => :id_gsn
     :id => :institution_id

In [43]:
df_inst_place_db = 
rightjoin(df_inst_place, select(df_mon_db, columns), on = :gsn_id => :id_gsn);

In [44]:
size(df_inst_place_db)

(5701, 20)

In [45]:
names(df_inst_place_db)

20-element Vector{String}:
 "id_monastery_location"
 "place_id"
 "relocated"
 "gsn_id"
 "location_begin_tpq"
 "location_begin_taq"
 "location_begin_note"
 "location_end_tpq"
 "location_end_taq"
 "location_end_note"
 "comment"
 "longitude"
 "latitude"
 "location_name"
 "main_location"
 "diocese_id"
 "geonames_id"
 "id"
 "place_name"
 "institution_id"

In [47]:
count(ismissing, df_inst_place_db.place_name), count(ismissing, df_inst_place_db.place_id)

(333, 12)

Etliche Klöster waren an mehreren Orten!

In [50]:
columns = [
    :institution_id => :institution_id,
    :id => :place_id,
    :place_name => :place_name,
    :location_begin_taq => :date_begin_taq,
    :location_begin_tpq => :date_begin_tpq,
    :location_end_taq => :date_end_taq,
    :location_end_tpq => :date_end_tpq,
    :location_begin_note => :note_begin,
    :location_end_note => :note_end
]

9-element Vector{Pair{Symbol, Symbol}}:
      :institution_id => :institution_id
                  :id => :place_id
          :place_name => :place_name
  :location_begin_taq => :date_begin_taq
  :location_begin_tpq => :date_begin_tpq
    :location_end_taq => :date_end_taq
    :location_end_tpq => :date_end_tpq
 :location_begin_note => :note_begin
   :location_end_note => :note_end

In [51]:
table_name = "institution_place";
Wds.filltable!(table_name, select(df_inst_place_db, columns))

┌ Info: Rows inserted: 5701
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


5701

Für alle Institutionen gibt es mindestens eine Eintrag in `institution_place`
``` sql
SELECT COUNT(*) FROM institution
WHERE id NOT IN (SELECT DISTINCT(institution_id) FROM institution_place);
```
Zahl der Institutionen für die mindestens eine `place_id` fehlt
``` sql
select count(distinct(institution_id)) from institution_place where place_id is null;
```
310

*2022-01-26* Es bleiben 152 Einträge ohne jegliche Zuordnung (`place_name`, `place_id`) übrig?!

Lege aus den Angaben für TPQ und TAQ einen Start- und Endzeitpunkt fest

In [10]:
table_name = "institution_place";
sql = "SELECT * FROM $(table_name)";
df_ip = Wds.sql_df(sql);

In [27]:
function estimate_date_begin(tpq, taq)
    margin = 20
    if !ismissing(tpq)
        return tpq
    end
    if !ismissing(taq)
        return taq - margin
    end
    return missing    
end

estimate_date_begin (generic function with 1 method)

In [26]:
function estimate_date_end(tpq, taq)
    margin = 20;
    if !ismissing(taq)
        return taq
    end
    if !ismissing(tpq)
        return tpq + margin
    end
    return missing    
end

estimate_date_end (generic function with 1 method)

In [25]:
transform!(df_ip, [:date_begin_tpq, :date_begin_taq] => ByRow(estimate_date_begin) => :num_date_begin);

In [28]:
transform!(df_ip, [:date_end_tpq, :date_end_taq] => ByRow(estimate_date_end) => :num_date_end);

In [39]:
table_update = "ip_update";
sql = "DROP TEMPORARY TABLE IF EXISTS $(table_update)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DROP TEMPORARY TABLE IF EXISTS ip_update", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [32]:
size(df_ip)

(5701, 13)

In [42]:
names(df_ip)

13-element Vector{String}:
 "id"
 "place_id"
 "place_name"
 "date_begin_tpq"
 "date_end_tpq"
 "institution_id"
 "comment"
 "date_begin_taq"
 "date_end_taq"
 "note_begin"
 "note_end"
 "num_date_begin"
 "num_date_end"

In [49]:
table_name = "institution_place"
step_msg = 800;
for (i, row) in enumerate(eachrow(df_ip))
    num_date_begin = ismissing(row[:num_date_begin]) ? "NULL" : row[:num_date_begin]
    num_date_end = ismissing(row[:num_date_end]) ? "NULL" : row[:num_date_end]
    sql = "UPDATE $(table_name) AS ip " *
    "SET  ip.num_date_begin = $(num_date_begin), ip.num_date_end = $(num_date_end)
    WHERE ip.id = $(row[:id]);"
    DBInterface.execute(Wds.dbwiag, sql)
    if i % step_msg == 0
        @info "Datensatz" i
    end
end

┌ Info: Datensatz
│   i = 800
└ @ Main In[49]:11
┌ Info: Datensatz
│   i = 1600
└ @ Main In[49]:11
┌ Info: Datensatz
│   i = 2400
└ @ Main In[49]:11
┌ Info: Datensatz
│   i = 3200
└ @ Main In[49]:11
┌ Info: Datensatz
│   i = 4000
└ @ Main In[49]:11
┌ Info: Datensatz
│   i = 4800
└ @ Main In[49]:11
┌ Info: Datensatz
│   i = 5600
└ @ Main In[49]:11


### Domstifte
Trage für die Domstifte einen Kurznamen (Ortsnamen ein)

In [10]:
table_name = "institution";
item_type_id = 3;
sql = "SELECT it.id_gsn, name, name_short
FROM $(table_name) AS it
JOIN item AS i on it.id = i.id
WHERE i.item_type_id = $(item_type_id)";
df_dst_name = Wds.sql_df(sql);

In [11]:
df_dst_name[1:7, :]

,id_gsn,name,name_short
,Int32?,String?,String?
1,226,Domstift Ratzeburg,missing
2,343,Domstift Bremen,missing
3,628,Domstift Münster,missing
4,616,Domstift Minden,missing
5,701,Prämonstratenser-Domstift Brandenburg an der Havel,missing
6,679,Domstift Schleswig,missing
7,676,Domstift Lübeck,missing


In [ ]:
name_short = [
    "Ratzeburg",
    "Bremen",
    "Münster",
    "Minden",
    "Brandenburg",
    "",
    "",
    "",
    "",
    "",
    "",
    "",
    "",
    "",
]